In [1]:
# https://ai.google/tools/datasets/google-facial-expression/
import imageio
import os
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm
import time
import numpy as np
import random
import hashlib
from skimage.transform import resize
import math
import pickle
import signal

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import * 
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
train_df = pd.read_csv('FEC_dataset/two_class_ready.csv')
train_df.head()
# hyperparams
SHAPE=(160,160,3)

In [16]:
def TripletGen(data, root='FEC_dataset/formatted', batch_size=128, rescale=1./255, shape=SHAPE[:2]):
    # yielding format must be [anchorlist, positlist, negatlist], dummy
    # 3 elements of shape (@batch_size,*@shape) in the list and an empty np.array of shape(@batch_size,1)
    
    ## the init code
    n = len(data)
    iterations = math.ceil(n/batch_size)
    print(f"Found {n} triplets in the dataset")
    
    def genFunc(batch_size = batch_size):
        y_dummy = np.empty((batch_size,))
        while True:
            for i in range(iterations):
                # use start and end as indices to work with
                start, end = i*batch_size, min((i+1)*batch_size, n)
                batch_size_eff = end - start

                arr_shape = (batch_size_eff, *shape, 3)
                anchor = np.empty(arr_shape)
                positive = np.empty(arr_shape)
                negative = np.empty(arr_shape)

                for arr_index in range(batch_size_eff):
                    data_index = arr_index + start

                    # anchor
                    path = os.path.join(root, data["Image1"][data_index])
                    temp = image.load_img( path, target_size = shape )
                    anchor[arr_index] = image.img_to_array(temp)*rescale

                    # positive
                    path = os.path.join(root, data["Image2"][data_index])
                    temp = image.load_img( path, target_size = shape )
                    positive[arr_index] = image.img_to_array(temp)*rescale

                    # negative
                    path = os.path.join(root, data["Image3"][data_index])
                    temp = image.load_img( path, target_size = shape )
                    negative[arr_index] = image.img_to_array(temp)*rescale

                yield [anchor, positive, negative], y_dummy[:batch_size]
    return genFunc(), iterations

imgs, iterations = TripletGen(train_df)
print("Before loop")
print("=="*18)

for i in range(5):
    left, right = next(imgs)
    for i, thing in enumerate(left):
        print(f"{i+1}th element: ",thing.shape)
    print("dummy", right.shape)
    print("=-"*18)
    
del(imgs)

Found 136379 triplets in the dataset
Before loop
1th element:  (128, 160, 160, 3)
2th element:  (128, 160, 160, 3)
3th element:  (128, 160, 160, 3)
dummy (128,)
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
1th element:  (128, 160, 160, 3)
2th element:  (128, 160, 160, 3)
3th element:  (128, 160, 160, 3)
dummy (128,)
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
1th element:  (128, 160, 160, 3)
2th element:  (128, 160, 160, 3)
3th element:  (128, 160, 160, 3)
dummy (128,)
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
1th element:  (128, 160, 160, 3)
2th element:  (128, 160, 160, 3)
3th element:  (128, 160, 160, 3)
dummy (128,)
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
1th element:  (128, 160, 160, 3)
2th element:  (128, 160, 160, 3)
3th element:  (128, 160, 160, 3)
dummy (128,)
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-


In [5]:
def batch_tf_triplet_loss(y_true, y_pred, alpha=0.4):
    total_lenght = y_pred.shape.as_list()[-1]
    anchor = y_pred[:,0:int(total_lenght*1/3)]
    positive = y_pred[:,int(total_lenght*1/3):int(total_lenght*2/3)]
    negative = y_pred[:,int(total_lenght*2/3):int(total_lenght*3/3)]
    
    sigma_pd = tf.math.reduce_sum(tf.norm(positive-anchor, axis=1))
    sigma_nd = tf.math.reduce_sum(tf.norm(negative-anchor, axis=1))
    loss = sigma_nd - sigma_pd + alpha
    return tf.maximum(loss, 0.0)

In [6]:
base_model = Sequential([
    #first convolution
    Conv2D(16, (3,3), activation='relu', input_shape=SHAPE),
    MaxPool2D(2,2),
    #second
    Conv2D(32, (3,3), activation='relu'),
    MaxPool2D(2,2),
    #third
    Conv2D(64, (3,3), activation='relu'),
    MaxPool2D(2,2),
    #fourth
    Conv2D(64, (3,3), activation='relu'),
    MaxPool2D(2,2),
    #fifth
    Conv2D(64, (3,3), activation='relu'),
    MaxPool2D(2,2),
    # hidden
    Flatten(),
    Dense(1024, activation='relu'),
    #Last layer
    Dense(16)
])
base_model.summary()
anchor_input = Input(SHAPE, name='anchor_input')
positive_input = Input(SHAPE, name='positive_input')
negative_input = Input(SHAPE, name='negative_input')

encoded_anchor = base_model(anchor_input)
encoded_positive = base_model(positive_input)
encoded_negative = base_model(negative_input)


merged_vector = concatenate([encoded_anchor, encoded_positive, encoded_negative], axis=-1, name='merged_layer')

train_model = Model(inputs=[anchor_input,positive_input, negative_input], outputs=merged_vector)
train_model.summary()

W0819 19:44:58.716810 140201793115968 deprecation.py:506] From /home/deepak/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 158, 158, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 79, 79, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 77, 77, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 38, 38, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 36, 36, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        3

In [7]:
train_model.compile(loss=batch_tf_triplet_loss, optimizer=SGD())
train, steps = TripletGen(train_df, batch_size=128)
checkpoint = ModelCheckpoint("emotion.hdf5", monitor='loss', verbose=1, save_best_only=True, mode='min')

Found 136379 triplets in the dataset


In [8]:
history = train_model.fit_generator(train, epochs=10, steps_per_epoch=steps, callbacks=[checkpoint])

Epoch 1/10


W0819 19:44:59.539559 140201793115968 deprecation.py:323] From /home/deepak/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


1065/1066 [============================>.] - ETA: 0s - loss: 0.0982
Epoch 00001: loss improved from inf to 0.09816, saving model to emotion.hdf5
1066/1066 [==============================] - 687s 645ms/step - loss: 0.0981
Epoch 2/10
1065/1066 [============================>.] - ETA: 0s - loss: 0.0000e+00
Epoch 00002: loss improved from 0.09816 to 0.00000, saving model to emotion.hdf5
1066/1066 [==============================] - 317s 298ms/step - loss: 0.0000e+00
Epoch 3/10
 322/1066 [========>.....................] - ETA: 3:42 - loss: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
# os.kill(os.getpid(), signal.SIGKILL)

In [11]:
train_model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [12]:
model = Model(inputs=[anchor_input], outputs=encoded_anchor)
model.load_weights("model.h5")
print("Loaded weights from h5 file")

Loaded weights from h5 file


In [35]:
train, steps = TripletGen(train_df[:1], batch_size=1)
images, _ = next(train)
anchor, positive, negative = tuple(images)
print(anchor.shape)
a,p,n = model.predict(anchor), model.predict(positive), model.predict(negative)
a.shape

Found 1 triplets in the dataset
(1, 160, 160, 3)


(1, 16)